In [45]:
### Library necessary to run this IPython Notebook
!pip install facebook-sdk
!pip install tqdm
!pip install requests
!pip install arrow

In [2]:
import pandas as pd
import facebook
import requests
import arrow
from tqdm import tqdm

In [3]:
ACCESS_TOKEN = 'EAACEdEose0cBAL0SUusAoNn51ZAF8kl412ttxz98SPjnHUlZBadgTpmKYY5gEsuvgIG0EGv2xeIKSWANug56CtQKwThHTIZBHDbunBZAIT4RU4x6sWj9pVd9dFfbCntSWhd8Lvk3ZCTtlYA2S5nJmdNIf4AbUZCjJxQDkNLyQJkTJm1x4ZAwWH6' # Get token...
PAGE_ID = 'CPeR014'

In [4]:
graph = facebook.GraphAPI(access_token = ACCESS_TOKEN, version = '2.7')

In [16]:
page = graph.get_object(
    id = PAGE_ID,
    fields='name, about, category, fan_count, feed.limit(25){created_time, permalink_url, type, status_type, message, shares}'
)

In [11]:
arw = arrow.utcnow()
current_month = arw.month

def extract_feed_data(page, rows = []):
    for post in tqdm(page['feed']['data']):
        likes = graph.get_object(str(post['id']) + '/likes?limit=0&summary=true')
        reactions = graph.get_object(str(post['id']) + '/reactions?limit=0&summary=true')
        comments = graph.get_object(str(post['id']) + '/comments?limit=0&summary=true')

        row_data = {
            'id': post['id'],
            'created_time': arrow.get(post['created_time']).format('YYYY-MM-DD HH:mm:ss'),
            'permalink_url': post['permalink_url'],
            'type': post['type'],
            'status_type': post['status_type'],
            'message': post['message'] if 'message' in post else '-',
            'total_likes': likes['summary']['total_count'],
            'total_reactions': reactions['summary']['total_count'],
            'total_comments': comments['summary']['total_count'],
            'total_shares': post['shares']['count'] if 'shares' in post else 0
        }
        rows.append(row_data)
    return pd.DataFrame(rows)
    
def extract_feed_data_current_month(feed, rows = []):
    should_finish = False

    for post in tqdm(feed['data'], 'Extracting current month data...'):
        post_month = arrow.get(post['created_time']).month
        if(post_month != current_month):
            should_finish = True
            break
        
        likes = graph.get_object(str(post['id']) + '/likes?limit=0&summary=true')
        reactions = graph.get_object(str(post['id']) + '/reactions?limit=0&summary=true')
        comments = graph.get_object(str(post['id']) + '/comments?limit=0&summary=true')

        row_data = {
            'id': post['id'],
            'created_time': arrow.get(post['created_time']).format('YYYY-MM-DD HH:mm:ss'),
            'permalink_url': post['permalink_url'],
            'type': post['type'],
            'status_type': post['status_type'],
            'message': post['message'] if 'message' in post else '-',
            'total_likes': likes['summary']['total_count'],
            'total_reactions': reactions['summary']['total_count'],
            'total_comments': comments['summary']['total_count'],
            'total_shares': post['shares']['count'] if 'shares' in post else 0
        }
        rows.append(row_data)

    if(should_finish):
        print('Month data complete!')
        return rows
    else:
        next_feed = requests.get(feed['paging']['next']).json()
        return extract_feed_data(next_feed, rows)

In [17]:
page_data = extract_feed_data(page)
page_data_month = pd.DataFrame(extract_feed_data_current_month(page['feed']))

Extracting current month data...:   0%|          | 0/25 [00:00<?, ?it/s]

Month data complete!


In [20]:
page_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 10 columns):
created_time       35 non-null object
id                 35 non-null object
message            35 non-null object
permalink_url      35 non-null object
status_type        35 non-null object
total_comments     35 non-null int64
total_likes        35 non-null int64
total_reactions    35 non-null int64
total_shares       35 non-null int64
type               35 non-null object
dtypes: int64(4), object(6)
memory usage: 2.8+ KB


In [21]:
page_data.head()

,created_time,id,message,permalink_url,status_type,total_comments,total_likes,total_reactions,total_shares,type
0,2017-02-15 16:44:22,340138559408803_1254316914657625,No último dia 19 de dezembro de 2016 o Capítul...,https://www.facebook.com/CPeR014/posts/1254316...,added_photos,0,9,9,0,photo
1,2017-01-11 11:49:50,340138559408803_825155040907150,-,https://www.facebook.com/CPeR014/posts/1221642...,added_photos,0,6,6,0,photo
2,2017-01-11 11:48:29,340138559408803_825151874240800,-,https://www.facebook.com/CPeR014/posts/1221641...,added_photos,0,22,22,0,photo
3,2016-11-01 20:21:40,340138559408803_1150259055063412,-,https://www.facebook.com/CPeR014/posts/1150256...,added_photos,0,4,4,0,photo
4,2016-11-01 20:08:10,340138559408803_915236991898954,-,https://www.facebook.com/CPeR014/posts/1150247...,added_photos,0,8,8,0,photo


In [22]:
from bokeh.charts import Scatter
from bokeh.io import output_notebook, show

In [23]:
# Informações que podemos tirar do gráfico seguinte: postagens com mais reações, quando ocorreram e seu tipo
p = Scatter(page_data, x='created_time', y='total_reactions', 
            xlabel='Created Time', ylabel='Total Reactions', 
            color='type', title='Created Time vs Total Reactions')
output_notebook()
show(p)
# Por via da dúvidas, der um zoom out

Loading BokehJS ...

In [ ]:
# References

# https://facebook-sdk.readthedocs.io/en/latest/api.html
# https://developers.facebook.com/docs/graph-api/reference/v2.7